In [1]:
%pip install duckdb numpy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
import kuzu
kuzu_path = r"C:\tempKuzu\ContainerLogistics.xml.kuzu"
query_names = {"kuzu": "auto-cypher-Kuzu.txt"}
result_names = {"kuzu": "kuzu-res.json"}
#queries = ['Q1','Q2','Q3','Q4','Q8','Q9','Q12','Q13']
queries = ['Q10']
#queries = ['Q10','Q11']
N = 10

_kuzu_db = None

def load_db(db):
    global _kuzu_db
    if db == "kuzu":
        if _kuzu_db is None:
            _kuzu_db = kuzu.Database(kuzu_path, read_only=True)
        return kuzu.Connection(_kuzu_db)

def run_query(con, sql, db):
    if db == "sqlite":
        return con.execute(sql).fetchall()
    if db == "duckdb":
        return con.execute(sql).fetchall()
    if db == "kuzu":
        result = con.execute(sql)
        return [tuple(row) for row in result]




In [2]:
import time
import numpy as np
import json

for db in ["kuzu"]:
    count = None
    for q in queries:
        durations = []
        with open(f"{q}/{query_names[db]}", "r", encoding="utf-8") as f:
            sql = f.read()
            con = load_db(db)

            for i in range(N):
                start = time.time()
                qres = run_query(con, sql, db)
                duration = time.time() - start
                count = len(qres)
                durations.append(duration)

        try:
            con.close()  
        except Exception:
            pass
        del con  

        with open(f"{q}/{result_names[db]}", "w", encoding="utf-8") as f:
            json.dump(durations, f)
        print(f"{db} for {q} counted {count} and took {1000*np.mean(durations)}")


kuzu for Q10 counted 428 and took 11.407303810119629
